# Multiple Sclerosis Classifier

This notebook contains a simple classifier for multiple sclerosis (MS) based on gene expression data. The goal of this project is to learn how to write a simple classifier for healthy and unhealthy patients.

The data is from a study comparing gene expression in peripheral blood samples between healthy human individuals and patients with relapsing-remitting multiple sclerosis (RRMS). The data was generated using RNA sequencing (RNA-Seq) to identify differences in mRNA levels between the two groups.

If you prefer you can use alternative datasets, classifying different diseases. Data can be acquired [here](https://www.ebi.ac.uk/gxa/experiments?experimentType=%22Differential%22&species=homo+sapiens). 
Choose datasets with at least two experimental factors. 
The default dataset (MS) was from [this](https://www.ebi.ac.uk/gxa/experiments/E-GEOD-66573/Results) link.
In case you are unsure how to handle different datasets, an alternative [dataset](https://www.ebi.ac.uk/gxa/experiments/E-ENAD-46/Results) is included as an example and can be loaded by changing `DATASET = lung` further below.


## Data Files

The data is located in the `data/` folder and consists of two datasets: Multiple Sclerosis (MS) and Lung.

### Multiple Sclerosis (MS)

The MS data is located in the `data/ms/` folder and consists of the following files:

*   `data/ms/raw-counts.tsv`: This file contains the raw gene counts for each sample. The rows represent genes and the columns represent samples.
*   `data/ms/experiment-design.tsv`: This file describes the samples, indicating which ones are from healthy controls and which are from RRMS patients, along with other relevant metadata.
*   `data/ms/query-results.tsv`: This file contains the query results from the Expression Atlas database.

### Lung

The Lung data is located in the `data/lung/` folder and consists of the following files:

*   `data/lung/raw-counts.tsv`: This file contains the raw gene counts for each sample. The rows represent genes and the columns represent samples.
*   `data/lung/experiment-design.tsv`: This file describes the samples and their metadata.
*   `data/lung/query-results.tsv`: This file contains the query results from the Expression Atlas database.

### Your Tasks

Your main task is to improve the classifier by selecting better features. This involves the following steps:

1.  **Familiarize yourself with the covariance matrix:** The covariance matrix is a powerful tool for understanding the relationships between features. The current version of this notebook generates a covariance matrix for the first 5 genes.
2.  **Normalization:** The data is normalized before calculating the covariance matrix. Why is this important? What would happen if you didn't normalize the data?
3.  **Feature Selection:** The current version of the notebook only uses the first 5 genes as features. You should experiment with different sets of features to see if you can improve the accuracy of the classifier. You can use the information from the `query results` dataframe to select genes that are differentially expressed between the two groups.
4.  **Interpret the covariance matrix:** What does it mean if a value in the covariance matrix is close to 0? What does it mean if it is close to 1? How can you use this information to select features?
5.  **Implement k-fold cross validation:** Since we have limited data (only 14 samples total), a single train/test split may not give us a reliable estimate of model performance. Implement k-fold cross validation to get a more robust evaluation of your classifier. Consider what value of k would be appropriate given our small dataset size. Compare the results from k-fold cross validation with the simple train/test split approach currently used in the notebook.

In [1]:
# Ensure that packages are installed
!pip install pandas scikit-learn matplotlib numpy

# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

RANDOM_STATE = 42


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### Raw Counts

This file contains the raw gene counts for each sample. The rows represent genes and the columns represent samples.

### Select the dataset to use

In [2]:
DATASET = "ms" # or "lung"

In [3]:
DATA_DIR = f"data/{DATASET}"

In [4]:
df_raw_counts = pd.read_csv(f"{DATA_DIR}/raw-counts.tsv", sep="\t", index_col=0)
df_raw_counts
df_filtered = df_raw_counts.loc[df_raw_counts.nunique(axis=1) > 1]

print(f"Original genes: {df_raw_counts.shape[0]}")
print(f"Filtered genes: {df_filtered.shape[0]}")

FileNotFoundError: [Errno 2] No such file or directory: 'data/ms/raw-counts.tsv'

### Experiment Design

This file describes the samples, indicating which ones are from healthy controls and which are from RRMS patients, along with other relevant metadata.

In [ ]:
df_experiment_design = pd.read_csv(f"{DATA_DIR}/experiment-design.tsv", sep="\t", index_col=0)
df_experiment_design

### Query Results

This file contains the query results from the Expression Atlas database.

In [ ]:
df_query_results = pd.read_csv(f"{DATA_DIR}/query-results.tsv", sep="\t", index_col=0)
df_query_results

### Data Preparation

Before we can train a classifier, we need to prepare the data. This involves selecting features and labels from the dataframes.

In [ ]:
def get_features_of_patient(df, patient_id):
    """
    Get the raw gene counts of a specific patient from the DataFrame.
    
    Parameters:
    df (pd.DataFrame): DataFrame containing patient data.
    patient_id (str): ID of the patient to retrieve features for. You can find the IDs in the first column of the analytics dataframe.
    
    Returns:
    pd.Series: Features of the specified patient.
    """
    return df[patient_id][:5].to_list() # TODO: Select features smartly, not just the first 5 (genes)

get_features_of_patient(df_raw_counts, "SRR1839791")

In [ ]:
def get_dataset(df_raw_counts, df_experiment_design): 
    """
    Get the entire dataset of features and labels.
    
    Parameters:
    df_raw_counts (pd.DataFrame): DataFrame containing raw gene counts.
    df_experiment_design (pd.DataFrame): DataFrame containing experiment design data.
    
    Returns:
    tuple: Features and labels for the dataset.
    """
    patient_ids = df_experiment_design.index.to_list()
    features = [get_features_of_patient(df_raw_counts, patient_id) for patient_id in patient_ids]
    labels = df_experiment_design["Sample Characteristic[disease]"].to_list()
    unique_labels = list(set(labels))
    labels = [unique_labels.index(label) for label in labels] # Convert labels to numbers
    return features, labels

features, labels = get_dataset(df_raw_counts, df_experiment_design)

In [ ]:
def select_top_genes(df, samples, labels, n_genes=50):
    healthy = labels == 0
    disease = labels == 1
    
    mean_h = df.loc[:, samples[healthy]].mean(axis=1)
    mean_d = df.loc[:, samples[disease]].mean(axis=1)
    
    return (mean_d - mean_h).abs().nlargest(n_genes).index


def build_dataset(df, genes, samples):
    return df.loc[genes, samples].T.values


def scale_and_pca(X_train, X_test, n_components=4):
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    pca = PCA(n_components=n_components)
    return pca.fit_transform(X_train), pca.transform(X_test), pca

### Train/Test Split

We split the data into a training set and a testing set. The training set is used to train the classifier, and the testing set is used to evaluate its performance. The main bottleneck we have with that project is the lack of enough data. In fact, we have 14 cases in total with 8 and 6 cases respectively. 

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.5, random_state=2)

### Train the Classifier

We use a simple Decision Tree Classifier to train the model.

In [ ]:
classifier = DecisionTreeClassifier(random_state=2)
classifier = classifier.fit(train_features, train_labels)

### Evaluate the Classifier

We use the accuracy score to evaluate the performance of the classifier.

In [ ]:
predicted_test_labels = classifier.predict(test_features)

In [ ]:
X = df_filtered.T.values
y = (df_experiment_design["Sample Characteristic[disease]"] != "normal").astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=RANDOM_STATE
)

model = DecisionTreeClassifier(random_state=RANDOM_STATE)
model.fit(X_train, y_train)

print("Baseline accuracy:", accuracy_score(y_test, model.predict(X_test)))

In [ ]:
k_values = [10, 20, 30, 50, 80, 100]
acc_no_pca, acc_pca = [], []

for k in k_values:
    selector = SelectKBest(f_classif, k=k)
    Xk = selector.fit_transform(X_train, y_train)
    Xk_test = selector.transform(X_test)
    
    Xk, Xk_test, _ = scale_and_pca(Xk, Xk_test, n_components=4)
    
    clf = DecisionTreeClassifier(random_state=RANDOM_STATE)
    clf.fit(Xk, y_train)
    acc_pca.append(accuracy_score(y_test, clf.predict(Xk_test)))

plt.figure(figsize=(8,5))
plt.plot(k_values, acc_pca, marker="o")
plt.xlabel("Number of Selected Genes")
plt.ylabel("Accuracy")
plt.title("Accuracy vs Top-k Gene Selection")
plt.grid(True)
plt.show()

In [ ]:
def get_single_gene_feature(df_raw_counts, df_experiment_design, gene): 
    """
    Get the single gene feature for all patients in the dataset.
    
    Parameters:
    df_raw_counts (pd.DataFrame): DataFrame containing raw gene counts.
    df_experiment_design (pd.DataFrame): DataFrame containing experiment design data.
    
    Returns:
    tuple: Features and labels for the dataset.
    """
    patient_ids = df_experiment_design.index.to_list()

    gene_index = df_raw_counts.index.get_loc(gene)

    features = [df_raw_counts[patient_id][gene_index:gene_index + 1].to_list() for patient_id in patient_ids]
    labels = df_experiment_design["Sample Characteristic[disease]"].to_list()
    unique_labels = list(set(labels))
    labels = [unique_labels.index(label) for label in labels] # Convert labels to numbers
    return features, labels

single_feature, labels = get_single_gene_feature(df_raw_counts, df_experiment_design, "ENSG00000250696")

In [ ]:
models = {
    "Decision Tree": DecisionTreeClassifier(random_state=RANDOM_STATE),
    "Logistic Regression": LogisticRegression(solver="liblinear"),
    "KNN (k=3)": KNeighborsClassifier(3),
    "Linear SVM": SVC(kernel="linear")
}

kf = KFold(n_splits=7, shuffle=True, random_state=RANDOM_STATE)
results = {}

samples = df_experiment_design.index.to_numpy()
labels = y.to_numpy()

for name, model in models.items():
    scores = []
    for train_idx, test_idx in kf.split(samples):
        genes = select_top_genes(df_raw_counts, samples[train_idx], labels[train_idx])
        X_train = build_dataset(df_raw_counts, genes, samples[train_idx])
        X_test = build_dataset(df_raw_counts, genes, samples[test_idx])
        
        X_train, X_test, _ = scale_and_pca(X_train, X_test)
        model.fit(X_train, labels[train_idx])
        scores.append(accuracy_score(labels[test_idx], model.predict(X_test)))
    
    results[name] = (np.mean(scores), np.std(scores))

In [ ]:
names = list(results.keys())
means = [results[n][0] for n in names]
stds = [results[n][1] for n in names]

plt.figure(figsize=(8,5))
plt.bar(names, means, yerr=stds, capsize=5)
plt.ylabel("Mean Accuracy")
plt.title("Model Comparison (7-Fold CV)")
plt.xticks(rotation=30)
plt.ylim(0,1)
plt.grid(axis="y", alpha=0.4)
plt.show()

In [ ]:
genes = select_top_genes(df_raw_counts, samples, labels)
X = build_dataset(df_raw_counts, genes, samples)

scaler = StandardScaler()
X = scaler.fit_transform(X)

pca = PCA(n_components=4)
X_pca = pca.fit_transform(X)

pairs = [(0,1), (0,2), (1,2)]

plt.figure(figsize=(12,4))
for i, (a,b) in enumerate(pairs):
    plt.subplot(1,3,i+1)
    for lab, color in zip([0,1], ["blue","red"]):
        plt.scatter(X_pca[labels==lab,a], X_pca[labels==lab,b], color=color, label=str(lab), alpha=0.7)
    plt.xlabel(f"PC{a+1}")
    plt.ylabel(f"PC{b+1}")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
cov = np.cov(X, rowvar=False)

plt.figure(figsize=(6,5))
plt.imshow(cov, cmap="viridis")
plt.colorbar()
plt.title("Covariance Matrix (Top 50 Genes)")
plt.tight_layout()
plt.show()

In [ ]:
top10 = genes[:10]
expr = np.log1p(df_raw_counts.loc[top10, samples].T)
expr["Disease"] = df_experiment_design["Sample Characteristic[disease]"].values

expr_melt = expr.melt(id_vars="Disease", var_name="Gene", value_name="Expression")

plt.figure(figsize=(12,5))
sns.stripplot(data=expr_melt, x="Gene", y="Expression", hue="Disease", jitter=True)
plt.xticks(rotation=45)
plt.title("Top 10 Differentially Expressed Genes")
plt.tight_layout()
plt.show()

In [ ]:
k_values = [10, 20, 30, 50, 100]
acc_no_pca, acc_pca = [], []

for k in k_values:
    selector = SelectKBest(f_classif, k=k)
    Xk = selector.fit_transform(X_train, y_train)
    Xk_test = selector.transform(X_test)

    scaler = StandardScaler()
    Xk = scaler.fit_transform(Xk)
    Xk_test = scaler.transform(Xk_test)

    # No PCA
    clf = DecisionTreeClassifier(random_state=42)
    clf.fit(Xk, y_train)
    acc_no_pca.append(accuracy_score(y_test, clf.predict(Xk_test)))

    # With PCA
    pca = PCA(n_components=4)
    clf.fit(pca.fit_transform(Xk), y_train)
    acc_pca.append(accuracy_score(y_test, clf.predict(pca.transform(Xk_test))))

plt.figure(figsize=(8,5))
plt.plot(k_values, acc_no_pca, marker='o', label="No PCA")
plt.plot(k_values, acc_pca, marker='s', label="With PCA")
plt.xlabel("Top-k Genes")
plt.ylabel("Accuracy")
plt.title("ANOVA Feature Selection: PCA vs No PCA")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
cv_splits = [5, 6, 7, 8, 9]
means, stds = [], []

for k in cv_splits:
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    scores = []

    for train_idx, test_idx in kf.split(samples):
        genes = select_top_genes(df_raw_counts, samples[train_idx], labels[train_idx])
        X_train = build_dataset(df_raw_counts, genes, samples[train_idx])
        X_test = build_dataset(df_raw_counts, genes, samples[test_idx])

        X_train, X_test, _ = scale_and_pca(X_train, X_test)
        model = SVC(kernel="linear")
        model.fit(X_train, labels[train_idx])
        scores.append(accuracy_score(labels[test_idx], model.predict(X_test)))

    means.append(np.mean(scores))
    stds.append(np.std(scores))

plt.figure(figsize=(7,5))
plt.bar([f"{k}-Fold" for k in cv_splits], means, yerr=stds, capsize=5)
plt.ylabel("Mean Accuracy")
plt.title("Effect of CV Splits on Model Stability")
plt.ylim(0,1)
plt.grid(axis="y", alpha=0.4)
plt.show()

In [ ]:
svm = SVC(kernel="linear")
svm.fit(X_pca, labels)

pc_importance = np.abs(svm.coef_[0])
explained = pca.explained_variance_ratio_

plt.figure(figsize=(8,4))
plt.bar(range(1, len(explained)+1), explained, alpha=0.6, label="Explained Variance")
plt.plot(range(1, len(pc_importance)+1), pc_importance, marker='o', color='red', label="PC Importance")
plt.xlabel("Principal Component")
plt.title("Explained Variance vs PC Importance")
plt.legend()
plt.tight_layout()
plt.show()